#  Fazendo o processamento do arquivo tvaberta_program_audience.csv
### Importando as *bibliotecas*

In [1]:
!pip install pandas

import pandas as pd
import datetime,os
from google.oauth2 import service_account

###
file_audiencia = "tvaberta_program_audience.csv"
path_audiencia = "gs://bucket-for-teste/pre-processar/" + file_audiencia

file_inventory = "tvaberta_inventory_availability.csv"
path_inventory = "gs://bucket-for-teste/pre-processar/" + file_inventory

file_resultado_final = "resultado.csv"
path_resultado_final = "gs://bucket-for-teste/bq-resultado/" + file_resultado_final

path_backup = "gs://bkp-resultado-bq-tv-aberta/backup"

!gsutil -m cp {path_audiencia} .
!gsutil -m cp {path_inventory} .


ModuleNotFoundError: No module named &#39;pandas&#39;

### Metodo para passar a data para dia da semana.


In [5]:
def get_weekend_v1(YYYY_MM_DD):
  year=YYYY_MM_DD.split('-')[0]
  mouth=YYYY_MM_DD.split('-')[1]
  day=YYYY_MM_DD.split('-')[2]
  today = datetime.datetime(int(year), int(mouth), int(day))
  return today.weekday()

def get_weekend_v2(DD_MM_AAAA):
  year=DD_MM_AAAA.split('/')[2]
  mouth=DD_MM_AAAA.split('/')[1]
  day=DD_MM_AAAA.split('/')[0]
  today = datetime.datetime(int(year), int(mouth), int(day))
  return today.weekday()
  

### Carregar o arquivo csv.

In [6]:

df = pd.read_csv(file_audiencia)
df

,signal,program_code,exhibition_date,program_start_time,average_audience
0,SP1,PTV1,2020-06-05,2020-06-05T15:01:00.000Z,9.338920e+05
1,SP1,VALE,2020-06-24,2020-06-24T19:36:00.000Z,1.835375e+06
2,BH,PTV1,2020-05-23,2020-05-23T15:01:00.000Z,2.874812e+05
3,DF,AUTO,2020-05-03,2020-05-03T12:14:00.000Z,8.649481e+04
4,DF,FGGE,2020-05-03,2020-05-03T18:46:00.000Z,1.521656e+05
...,...,...,...,...,...
8982,NAC,N18H,2020-05-04,2020-05-04T21:31:00.000Z,2.093098e+07
8983,NAC,JHOJ,2020-05-13,2020-05-13T16:25:00.000Z,1.331286e+07
8984,NAC,SHOV,2020-06-07,2020-06-07T15:30:00.000Z,8.610939e+06
8985,NAC,SHOV,2020-05-17,2020-05-17T15:30:00.000Z,1.056899e+07


### Criar uma nova coluna chamado weekday.


In [7]:
df['weekday'] = pd.Series(df.exhibition_date.apply(get_weekend_v1), index=df.index)

### Ordenar as colunas.


In [8]:
df.loc[:,["signal","program_code","exhibition_date","weekday","average_audience"]]

,signal,program_code,exhibition_date,weekday,average_audience
0,SP1,PTV1,2020-06-05,4,9.338920e+05
1,SP1,VALE,2020-06-24,2,1.835375e+06
2,BH,PTV1,2020-05-23,5,2.874812e+05
3,DF,AUTO,2020-05-03,6,8.649481e+04
4,DF,FGGE,2020-05-03,6,1.521656e+05
...,...,...,...,...,...
8982,NAC,N18H,2020-05-04,0,2.093098e+07
8983,NAC,JHOJ,2020-05-13,2,1.331286e+07
8984,NAC,SHOV,2020-06-07,6,8.610939e+06
8985,NAC,SHOV,2020-05-17,6,1.056899e+07


#### Fazer cópia do dataframe 


In [9]:
novo_df = df.loc[:,["signal","program_code","exhibition_date","weekday","average_audience"]].copy()

### Ordenar dataframe por exhibition_date


In [10]:
data_sort = novo_df.sort_values(['exhibition_date'], ascending=False)

### Group as colunas para fazer a mediana, pegando os últimos quatros registros do dia da semana.


In [11]:
data_gby = data_sort.groupby(['signal','program_code','weekday']).head(4) 

### Calcula a mediana


In [12]:
df_audience = data_gby.groupby(['signal','program_code','weekday'], as_index=True).median().round({'average_audience': 2})
df_audience

average_audience
signal program_code weekday                  
BH     ACDA         5               460454.20
       ALTA         5               201008.82
       AUTO         6               197990.32
       BIAL         0               113326.74
                    1                90521.87
...                                       ...
SP1    VALE         3              1827112.19
                    4              1856927.41
       VOIK         6               994919.87
       ZORR         5              1693933.70
       _999         5               717788.87

[977 rows x 1 columns]

### Export o dataframe csv

In [13]:
df_audience.info()

&lt;class &#39;pandas.core.frame.DataFrame&#39;&gt;
MultiIndex: 977 entries, (&#39;BH&#39;, &#39;ACDA&#39;, 5) to (&#39;SP1&#39;, &#39;_999&#39;, 5)
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   average_audience  977 non-null    float64
dtypes: float64(1)
memory usage: 11.5+ KB




---



---



---



# Carregar o arquivo do inventroy availability


In [15]:
df_inventory = pd.read_csv(file_inventory, sep=";")


### Adicionar a coluna do weekday

In [16]:
df_inventory['weekday'] = pd.Series(df_inventory.date.apply(get_weekend_v2))
df_inventory

,signal,program_code,date,available_time,weekday
0,SP1,HUCK,25/07/2020,10,5
1,SP1,HUCK,01/08/2020,300,5
2,SP1,HUCK,08/08/2020,118,5
3,SP1,HUCK,15/08/2020,106,5
4,SP1,HUCK,22/08/2020,6,5
...,...,...,...,...,...
2287,BH,NBRA,24/08/2020,218,0
2288,BH,NBRA,25/08/2020,66,1
2289,BH,NBRA,26/08/2020,44,2
2290,BH,NBRA,27/08/2020,190,3


In [17]:
#df_inventory = df_inventory.groupby(['signal','program_code','weekday'],as_index=True).sum()
#df_inventory

In [18]:
#df_inventory.to_csv('tratados-'+file_inventory)


# Tratamendo do merge dos dataframes

### Verificar o dataframe do audiences

In [19]:
df_audience

average_audience
signal program_code weekday                  
BH     ACDA         5               460454.20
       ALTA         5               201008.82
       AUTO         6               197990.32
       BIAL         0               113326.74
                    1                90521.87
...                                       ...
SP1    VALE         3              1827112.19
                    4              1856927.41
       VOIK         6               994919.87
       ZORR         5              1693933.70
       _999         5               717788.87

[977 rows x 1 columns]

### Verificar o dataframe do inventory

In [20]:
df_inventory

,signal,program_code,date,available_time,weekday
0,SP1,HUCK,25/07/2020,10,5
1,SP1,HUCK,01/08/2020,300,5
2,SP1,HUCK,08/08/2020,118,5
3,SP1,HUCK,15/08/2020,106,5
4,SP1,HUCK,22/08/2020,6,5
...,...,...,...,...,...
2287,BH,NBRA,24/08/2020,218,0
2288,BH,NBRA,25/08/2020,66,1
2289,BH,NBRA,26/08/2020,44,2
2290,BH,NBRA,27/08/2020,190,3


### Fazer o merge dos dataframes

In [21]:
result = pd.merge(df_audience, df_inventory, on=['signal','program_code','weekday'])
result

,signal,program_code,weekday,average_audience,date,available_time
0,BH,AUTO,6,197990.32,26/07/2020,140
1,BH,AUTO,6,197990.32,02/08/2020,94
2,BH,AUTO,6,197990.32,09/08/2020,38
3,BH,AUTO,6,197990.32,16/08/2020,262
4,BH,AUTO,6,197990.32,23/08/2020,296
...,...,...,...,...,...,...
1975,SP1,VALE,4,1856927.41,31/07/2020,190
1976,SP1,VALE,4,1856927.41,07/08/2020,226
1977,SP1,VALE,4,1856927.41,14/08/2020,14
1978,SP1,VALE,4,1856927.41,21/08/2020,284


### Salvar o novo dataframe

In [22]:
result.to_csv(file_resultado_final, index=False)

# Fazer backup do velho e enviar resposta para o bucket

In [23]:
import time

millis = int(round(time.time() * 1000))

!gsutil -m mv {path_resultado_final} {path_backup}-{millis}.csv

!gsutil -m cp {file_resultado_final} {path_resultado_final}


Copying gs://bucket-for-teste/bq-resultado/resultado.csv [Content-Type=text/csv]...
Removing gs://bucket-for-teste/bq-resultado/resultado.csv...

Operation completed over 1 objects/68.0 KiB.                                     
Copying file://resultado.csv [Content-Type=text/csv]...

Operation completed over 1 objects/68.0 KiB.                                     


# Reprocessar BigQuery

In [24]:
nome_tabela_bq = "dataset_01.resultado"

!bq rm -f -t {nome_tabela_bq}

!bq load --quote "" --autodetect --format=csv {nome_tabela_bq} {path_resultado_final}

Waiting on bqjob_r5871e07c32e70746_00000174f93a7bc7_1 ... (1s) Current status: DONE   
